In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import PIL 
from tqdm import tqdm
import matplotlib.pyplot as plt
# tsne and pca
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from DeepTaxonNet import DeepTaxonNet
import argparse
import utils

from sklearn.mixture import GaussianMixture
import os
import sys

In [4]:
train_loader, test_loader, train_set, test_set = utils.get_data_loader('fashion-mnist', 128, False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 276kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.35MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.96MB/s]
/nethome/zwang910/miniconda3/envs/dl/lib/python3.13/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

In [6]:
n_layers=10
model = DeepTaxonNet(
    n_layers=n_layers,
    enc_hidden_dim=128*1*1,
    dec_hidden_dim=(128,1,1),
    input_dim=1*28*28,
    latent_dim=10,
    encoder_name='omniglot',
    decoder_name='omniglot',
    kl1_weight=1
).to(device)

path = './models/'
model_name = 'dtn-10-fmnist.pt'
model.load_state_dict(torch.load(f'{path}{model_name}'))

<All keys matched successfully>

# Accuracy

In [7]:
annotation = utils.label_annotation(model, train_loader, 10, device)
acc = utils.basic_node_evaluation(model, annotation, test_loader, device)
print('acc:', acc)

acc: 0.8449


# NMI

In [8]:
nmi = utils.compute_nmi(model, annotation, test_loader, device)
print(f"MNI: {nmi}")

MNI: 0.7595240007583479


# DP

In [ ]:
dendrogram_purity = utils.compute_soft_dendrogram_purity_test_only(model, test_loader, device)
print('dendrogram_purity:', dendrogram_purity)

Processing test data to get probability distributions (pcx)...


Evaluating Test Set: 100%|██████████| 40/40 [00:01<00:00, 21.84it/s]


Processed 10000 test samples. Found 10 classes and 2047 nodes.
Calculating node purities based on test set expected counts...
Node purities calculated.
Calculating Soft Dendrogram Purity (iterating over pairs)...


Processing Classes: 100%|██████████| 10/10 [07:59<00:00, 47.98s/it]

Calculation complete.
dendrogram_purity: 0.6167082260712516


# LP

In [40]:
overall_leaf_purity, per_leaf_purities = utils.compute_leaf_purity(model, test_loader, device)
print('overall_leaf_purity:', overall_leaf_purity)
print('per_leaf_purities:', per_leaf_purities)

overall_leaf_purity: 0.8231987092223002
per_leaf_purities: {0: (0.6359920726636731, 1.5872981225024254), 1: (0.6317036023522133, 2.8059586161639283), 2: (0.9999740074923239, 8.099182762385055), 3: (0.9999999986809572, 8.667099029056224), 4: (0.7585972433982828, 4.01482337388674), 5: (0.5592874261214941, 5.265802571773249), 6: (0.7424438334401076, 2.4655320508363014), 7: (0.47055860697313767, 1.8444374236739862), 8: (0.9571660202529276, 5.909256754932587), 9: (0.7328355506594745, 8.303666567209966), 10: (0.8033865286366191, 6.428504915969903), 11: (0.4223712218527423, 5.195457226838208), 12: (0.6875316787545459, 4.68384601727734), 13: (0.795217745393234, 7.271334832141424), 14: (0.5205225338774863, 3.970943525541906), 15: (0.8337953999911526, 7.318100871609492), 16: (0.5797231021122199, 1.9121550748380822), 17: (0.5133208482073255, 3.5512397768429023), 18: (0.9954028630039613, 4.738190919742339), 19: (0.8682790131038614, 4.622406934367384), 20: (0.548532291939444, 2.299295943160373), 21